In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras import callbacks


TensorFlow version: 2.2.0


In [2]:
test = np.zeros((212, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\Testing_1696-1908.npy", 'rb') as f1:
        test = np.load(f1)
        

In [3]:
test = np.reshape(test, (212, 340, 444, 6))
yTest = np.zeros((212))
for i in range (212):
    if (i%2==0):
        yTest[i] = 0
    else:
        yTest[i] = 1


In [4]:
print (test.shape)
img_shape = (340, 444, 6)


(212, 340, 444, 6)


In [5]:
unitsChoices = [16, 32, 64, 128, 256]
poolSizes = [3, 5]
batchSizes = [4, 8, 16, 32]
nbEpochs = [9, 10, 11, 12]
learnRates = [1e-3, 1e-4, 1e-5]

In [6]:
img = layers.Input(shape=img_shape)

conv1 = layers.Conv2D(unitsChoices[4], (poolSizes[1], poolSizes[1]), padding='same', activation='relu', strides=5)(img)
bn1 = layers.BatchNormalization()(conv1)
maxPooling1 = layers.MaxPooling2D(padding='same')(bn1)

conv2 = layers.Conv2D(unitsChoices[2], (poolSizes[1], poolSizes[1]), padding='same', activation='elu', strides=5)(maxPooling1)
bn2 = layers.BatchNormalization()(conv2)
maxPooling2 = layers.MaxPooling2D(padding='same')(bn2)

conv3 = layers.Conv2D(unitsChoices[2], (poolSizes[1], poolSizes[1]), padding='same', activation='elu', strides=3)(maxPooling2)
bn3 = layers.BatchNormalization()(conv3)
maxPooling3 = layers.MaxPooling2D(padding='same')(bn3)

conv4 = layers.Conv2D(unitsChoices[4], (poolSizes[0], poolSizes[0]), padding='same', activation='elu', strides=3)(maxPooling3)
bn4 = layers.BatchNormalization()(conv4)
maxPooling4 = layers.MaxPooling2D(padding='same')(bn4)

conv5 = layers.Conv2D(unitsChoices[1], (poolSizes[0], poolSizes[0]), padding='same', activation='elu', strides=3)(maxPooling4)
maxPooling5 = layers.MaxPooling2D(padding='same')(conv5)

flatten = layers.Flatten()(maxPooling5)

dense1 = layers.Dense(unitsChoices[4], activation = "relu")(flatten)
drop2 = layers.Dropout(0.5420313649456072)(dense1)
res = layers.Dense(1, activation='sigmoid')(drop2)

model = Model(img, res)
opt = tf.keras.optimizers.RMSprop(learning_rate=learnRates[2])
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [7]:
model.load_weights("Model2TrainingB-HO-V2.hdf5")

In [8]:
yPred = model.predict(test)

In [9]:
yPred[yPred>=0.5] = 1
yPred[yPred<0.5] = 0
yPred = yPred.astype(int)

In [10]:
yPred = yPred.flatten()
print (yPred)

[0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1
 1 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0
 0 0 1 0 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1
 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1]


In [11]:
correct = 0
for i in range (212):
    if (yPred[i]==yTest[i]):
        correct+=1
        
print (correct/212)

0.7547169811320755
